# 拉勾网招聘信息爬取

### 1、导包

In [1]:
# -*- coding:utf-8 -*-
import re,json,math,datetime
from urllib import request
import pandas as pd

### 2、字符串处理函数

In [2]:
def ProcessingString(string):
    string = string.encode('utf-8')
    string = str(string).replace(r'\x','%').replace(r"'","")
    string = re.sub('^b','',string)
    return string

### 3、计算搜索内容的数量与页数
* 每页默认显示数量为15项
* 数据使用Ajax，其json格式经常改动

In [3]:
def SearchPageCount(position, city):
    positionTemp = ProcessingString(position)
    cityTemp = ProcessingString(city)
    i = 0
    url = 'http://www.lagou.com/jobs/positionAjax.json?city='+cityTemp+'&needAddtionalResult=false&first=false'+'&kd='+positionTemp+'&pn='+str(i+1)
    with request.urlopen(url) as f:
        data = f.read()
        totalCount = int(json.loads(str(data,encoding='utf-8',errors='ignore'))["content"]['positionResult']["totalCount"])
        count = math.ceil(totalCount/15) # 向下取整
        print('本次搜索到%d个职位'%totalCount)
        print('本次搜索共%d页'%count)
    return count

### 4、爬取函数
* 抓取内容
* 数据存储

In [4]:
def LaGouSpiderWithKeyWord(position, city):
    
    positionTemp = ProcessingString(position)
    cityTemp = ProcessingString(city)
    # 获取总共页数
    pageCount = SearchPageCount(positionTemp,cityTemp)
    n=1
    rdata=pd.DataFrame()
    # 遍历每一页
    for i in range(0,pageCount):           
        # url生成，并获取内容
        url = 'http://www.lagou.com/jobs/positionAjax.json?city='+cityTemp+'&needAddtionalResult=false&first=false'+'&kd='+positionTemp+'&pn='+str(i+1)
        data = request.urlopen(url).read()
    
        # 读取Json数据/解析并筛选
        jsondata = json.loads(str(data,encoding='utf-8',errors='ignore'))['content']['positionResult']['result']
        # 遍历当页的所用position职位
        for j in range(0,len(jsondata)):
            # 打印
            # print(str(n)+" "+jsondata[j]['positionName']+" "+jsondata[j]['companyShortName']+" "+jsondata[j]['salary'])
            # n = n+1
                
            #将每一行数据做成Series，之后再合并
            if j == 0:
                rdata=pd.DataFrame(pd.Series(data=jsondata[j])).T
            else:
                rdata=pd.concat([rdata,pd.DataFrame(pd.Series(data=jsondata[j])).T])
                    
        #构造totaldata，是所有页面的集合，rdata是这一个页面的集合
        if i == 0:
             totaldata=rdata
        else:
             totaldata=pd.concat([totaldata,rdata])
                            
    #获取爬取结束时间
    now = datetime.datetime.now().strftime('%m-%d')
     
    #开始写入数据库
    totaldata.to_excel(position+'_'+city+'_'+now+'.xls',sheet_name='sheet1')

In [ ]:
now = datetime.datetime.now()
print(str(now))
now.strftime('%Y-%m-%d_%H:%M')
now.strftime('%y-%m-%d_%H:%M')

### 4、执行

获取"数据分析","深圳"所有的招聘信息

In [ ]:
LaGouSpiderWithKeyWord(u"数据分析",u"深圳")

In [ ]:
LaGouSpiderWithKeyWord(u"数据分析",u"全国")

In [ ]:
LaGouSpiderWithKeyWord(u"数据挖掘",u"全国")

In [6]:
LaGouSpiderWithKeyWord(u"数据",u"全国")

本次搜索到6821个职位
本次搜索共455页


获取"数据分析","全国"招聘数量

In [ ]:
SearchPageCount(u"数据分析",u"全国")

In [5]:
SearchPageCount(u"数据",u"全国")

本次搜索到6821个职位
本次搜索共455页


455

---

#### 代码集合，勿动

import re,json
from urllib import request
import pandas as pd
import math

#### 处理字符串的函数
def ProcessingString(string):
    string = string.encode('utf-8')
    string = str(string).replace(r'\x','%').replace(r"'","")
    string = re.sub('^b','',string)
    return string

#### 计算数量，页数
def SearchPageCount(position, city):
    i = 1
    positionTemp = ProcessingString(position)
    cityTemp = ProcessingString(city)
    url = 'http://www.lagou.com/jobs/positionAjax.json?city='+cityTemp+'&needAddtionalResult=false&first=false'+'&kd='+positionTemp+'&pn='+str(i+1)
    with request.urlopen(url) as f:
        data = f.read()
        totalCount = int(json.loads(str(data,encoding='utf-8',errors='ignore'))["content"]['positionResult']["totalCount"])
        count = math.ceil(totalCount/15) # 向下取整
        print('本次搜索到%d个职位'%totalCount)
        print('本次搜索共%d页'%count)
    return count

#### 爬取&存储
def LaGouSpiderWithKeyWord(position, city):
    positionTemp = ProcessingString(position)
    cityTemp = ProcessingString(city)
    # 获取总共页数
    pageCount = SearchPageCount(positionTemp,cityTemp)
    n=1
    rdata=pd.DataFrame()
    # 遍历每一页
    for i in range(0,pageCount):           
        # url生成，并获取内容
        url = 'http://www.lagou.com/jobs/positionAjax.json?city='+cityTemp+'&needAddtionalResult=false&first=false'+'&kd='+positionTemp+'&pn='+str(i+1)
        data = request.urlopen(url).read()
    
        # 读取Json数据/解析并筛选
        jsondata = json.loads(str(data,encoding='utf-8',errors='ignore'))['content']['positionResult']['result']
        # 遍历当页的所用position职位
        for j in range(0,len(jsondata)):
            print(str(n)+" "+jsondata[j]['positionName']+" "
                    +jsondata[j]['companyShortName']+" "
                    +jsondata[j]['salary'])
            n = n+1
                
            #将每一行数据做成Series，之后再合并
            if j == 0:
                rdata=pd.DataFrame(pd.Series(data=jsondata[j]['position'])).T
            else:
                rdata=pd.concat([rdata,pd.DataFrame(pd.Series(data=jsondata[j]['position'])).T])
                    
        #构造totaldata，是所有页面的集合，rdata是这一个页面的集合
        if i == 0:
             totaldata=rdata
        else:
             totaldata=pd.concat([totaldata,rdata])
                            
                    
    # 开始写入数据库
    totaldata.to_excel('lagou.xls',sheet_name='sheet1')

---